In [1]:
import faiss

from uuid import uuid4
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [2]:
import sys
sys.path.append("/culture_backoffice_data/app")

from app.src.v1.db.mongo import connect_to_mongo, convert_objectid_to_str
from app.src.v1.rag.retriever.vector_retriever import VectorRetriever
from app.src.v1.rag.retriever.hybrid_retriever import HybridRetriever

from notebooks.eda import analyze_document_lengths, analyze_token_lengths

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Config

In [3]:
model_list = [
    "intfloat/multilingual-e5-large-instruct", 
    "dragonkue/BGE-m3-ko",
    "Alibaba-NLP/gte-multilingual-base",
    "Alibaba-NLP/gte-Qwen2-7B-instruct",
    "Salesforce/SFR-Embedding-2_R", 
]

In [4]:
cfg = {
    "retrieve_method": "semantic", ## semantic, hybrid
    "embed_provider": "huggingface",
    "embed_model_name": model_list[2],

    "top_k": 20,
    "vector_threshold": 0.7,
    "bm25_threshold": 0.5,

    "index_type": "IP",  # L2, IP, HNSW 
    "model_kwargs": {
        "device": "cuda:1",
        "trust_remote_code": True
    },
    "encode_kwargs": {
        "normalize_embeddings": False,
        "batch_size": 512,
        "multi_process": True,
        "show_progress": True
    },
    "hybrid_search": {
        "enabled": True,
        "method": "cc",  # "rrf" 또는 "cc"
        "rrf": {
            "k": 60  # RRF 상수 k
        },
        "cc": {
            "alpha": 0.5  # 벡터 검색 가중치 (1-alpha는 BM25 가중치)
        },
        "bm25": {
            "k1": 1.5,  # 용어 빈도 가중치
            "b": 0.75   # 문서 길이 정규화 매개변수
        }
    },
    "use_cache": True,  # 캐싱 활성화 여부
    "cache_dir": ".cache",  # 캐시 저장 디렉토리
    "tokenize_processes": 4  # 토큰화에 사용할 프로세스 수
}

## Data

### DB에서 데이터 꺼내기

In [5]:
company_collection = connect_to_mongo("culture_db", "company")
hompage_collection = connect_to_mongo("culture_db", "company_homepage")

# 기업 데이터 가져오기
print("기업 데이터 불러오는 중...")
companies = company_collection.find()
total_data = []
for company in companies:
    id = company["_id"]
    name = company["company_name"]
    biz_no = company["biz_no"]
    total_data.append((id, name, biz_no))

print(f"총 {len(total_data)}개 기업 데이터를 불러왔습니다.")

기업 데이터 불러오는 중...
총 60개 기업 데이터를 불러왔습니다.


In [9]:
doc_results = analyze_document_lengths(total_data, hompage_collection)

기업별 문서 길이 분석 중...


100%|██████████| 60/60 [00:00<00:00, 586.25it/s]

In [10]:
token_results = analyze_token_lengths(total_data, hompage_collection, model_list[2])

임베딩 모델 'Alibaba-NLP/gte-multilingual-base'의 토크나이저로 토큰화 분석 중...


100%|██████████| 60/60 [00:14<00:00,  4.06it/s]
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'defaul

In [11]:
print("\n분석이 완료되었습니다.")
print(f"분석 결과는 'analysis_results' 디렉토리에 저장되었습니다.")

# 요약 정보 출력
print("\n=== 문서 길이 분석 요약 ===")
print(f"총 문서 수: {doc_results['total_stats']['total_documents']}")
print(f"평균 문서 길이: {doc_results['total_stats']['avg_length']:.1f} 글자")
print(f"중앙값 문서 길이: {doc_results['total_stats']['median_length']:.1f} 글자")
print(f"최소 문서 길이: {doc_results['total_stats']['min_length']} 글자")
print(f"최대 문서 길이: {doc_results['total_stats']['max_length']} 글자")

print("\n=== 토큰 길이 분석 요약 ===")
print(f"모델: {cfg['embed_model_name']}")
print(f"평균 토큰 길이: {token_results['total_token_stats']['avg_token_length']:.1f} 토큰")
print(f"중앙값 토큰 길이: {token_results['total_token_stats']['median_token_length']:.1f} 토큰")
print(f"최소 토큰 길이: {token_results['total_token_stats']['min_token_length']} 토큰")
print(f"최대 토큰 길이: {token_results['total_token_stats']['max_token_length']} 토큰")
print(f"평균 토큰/문자 비율: {token_results['correlation_data']['token_to_char_ratio'].mean():.4f}")


분석이 완료되었습니다.
분석 결과는 'analysis_results' 디렉토리에 저장되었습니다.

=== 문서 길이 분석 요약 ===
총 문서 수: 7911
평균 문서 길이: 2347.8 글자
중앙값 문서 길이: 1492.0 글자
최소 문서 길이: 11 글자
최대 문서 길이: 116366 글자

=== 토큰 길이 분석 요약 ===
모델: Alibaba-NLP/gte-multilingual-base
평균 토큰 길이: 900.9 토큰
중앙값 토큰 길이: 598.0 토큰
최소 토큰 길이: 6 토큰
최대 토큰 길이: 42343 토큰
평균 토큰/문자 비율: 0.4460


In [ ]:
hompage_collection = connect_to_mongo("culture_db", "company_homepage")

total_pages = []
for id, name, biz_no in total_data:
    id = convert_objectid_to_str(id)
    print(f"{id}, {name} {biz_no}")
    homepage = hompage_collection.find_one({"company_id": id})

    if homepage is None:
        print(f"Warning: No homepage data found for {name}")
        continue

    pages = homepage["pages"]
    print(f"{name} : {len(pages)}")
    for page in homepage["pages"]:
        total_pages.append((name, page))

print(len(total_pages))

In [ ]:
def analyze_and_optimize_chunking(docs, sample_size=100):
    

In [ ]:
total_docs = []
for company_name, page in total_pages:
    total_docs.append(Document(page_content=page["text"], metadata={"company_name": company_name, "url": page["url"]}, id=uuid4()))

print(len(total_docs))

In [ ]:
sample = total_docs[0]
print("=" * 100)
print(sample.id, sample.metadata['url'], sample.metadata['company_name'])
print("=" * 100)
print(sample.page_content)

## Vector Retriever

In [ ]:
vector_store = VectorRetriever(total_docs, cfg)

In [ ]:
results = vector_store.similarity_search_with_score("코리아교육그룹의 비전 및 목표")

for document, score in results:
    doc_id = document.id
    corp_name = document.metadata['company_name']
    page_url = document.metadata['url']
    content = document.page_content

    print("=" * 100)
    print(f"{corp_name}, {page_url}, {score}")
    print("=" * 100)
    print(content, "\n\n")

In [ ]:
results = vector_store.similarity_search_with_score("코리아교육그룹의 일하는 방식과 조직문화")

for document, score in results:
    doc_id = document.id
    corp_name = document.metadata['company_name']
    page_url = document.metadata['url']
    content = document.page_content

    print("=" * 100)
    print(f"{corp_name}, {page_url}, {score}")
    print("=" * 100)
    print(content, "\n\n")